# 02 - Feature Engineering Notebook
**Goal:**  Clean the raw data, add more features, and create lagged/rolling averaged features.

- Implement a 1M, 3M, 6M, and 12M lag for economic features.

In [ ]:
# Load raw csv data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

combined = pd.read_csv("data/raw/combined.csv", index_col=0, parse_dates=True)
fred_only = pd.read_csv("data/raw/fred_only.csv", index_col=0, parse_dates=True)

df = combined.copy()


